In [1]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('WARN')
from absl import logging
logging.set_verbosity(logging.WARN)

In [2]:
spec = object_detector.EfficientDetSpec(
  model_name='efficientdet-lite0', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite0/feature-vector/1', 
  hparams={'max_instances_per_image': 8000}
)

In [3]:
import os

annotation_paths = [os.path.splitext(path)[0] for path in os.listdir("dataset/tf_labels")]
# TODO: Investigate how shuffling the various datasets affect model accuracy
test_annotations = annotation_paths[0:250]
validate_annotations = annotation_paths[250:500]
training_annotations = annotation_paths[500:]

In [4]:
train_data = object_detector.DataLoader.from_pascal_voc(
  images_dir="dataset/data", 
  annotations_dir="dataset/tf_labels", 
  annotation_filenames=training_annotations, 
  label_map={1: "Person", 2: "Horse"}
)
test_data = object_detector.DataLoader.from_pascal_voc(
  images_dir="dataset/data", 
  annotations_dir="dataset/tf_labels", 
  annotation_filenames=test_annotations, 
  label_map={1: "Person", 2: "Horse"}
)
validate_data = object_detector.DataLoader.from_pascal_voc(
  images_dir="dataset/data", 
  annotations_dir="dataset/tf_labels", 
  annotation_filenames=validate_annotations, 
  label_map={1: "Person", 2: "Horse"}
)

(train_data.size, test_data.size, validate_data.size)

INFO:tensorflow:Cache will be stored in /tmp/tmpe_j313es with prefix filename 02d31dbb50930ef4f9c6074b548954ab. Cache_prefix is /tmp/tmpe_j313es/02d31dbb50930ef4f9c6074b548954ab


INFO:tensorflow:Cache will be stored in /tmp/tmpe_j313es with prefix filename 02d31dbb50930ef4f9c6074b548954ab. Cache_prefix is /tmp/tmpe_j313es/02d31dbb50930ef4f9c6074b548954ab


INFO:tensorflow:On image 0


INFO:tensorflow:On image 0


INFO:tensorflow:On image 100


...

INFO:tensorflow:On image 100


INFO:tensorflow:On image 100


INFO:tensorflow:On image 200


INFO:tensorflow:On image 200


(1500, 250, 250)

In [5]:
model = object_detector.create(
  train_data, 
  model_spec=spec, 
  batch_size=8, 
  train_whole_model=True, 
  validation_data=validate_data
)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...
INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:target_size = (320, 320), output_size = (320, 320)
INFO:absl:LR schedule method: cosine
INFO:absl:Use SGD optimizer


Epoch 1/50
  6/187 [..............................] - ETA: 22s - det_loss: 1.8228 - cls_loss: 1.1199 - box_loss: 0.0141 - reg_l2_loss: 0.0630 - loss: 1.8858 - learning_rate: 0.0080 - gradient_norm: 0.8280WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1215s vs `on_train_batch_end` time: 0.1860s). Check your callbacks.


187/187 [==============================] - 51s 146ms/step - det_loss: 1.0832 - cls_loss: 0.5778 - box_loss: 0.0101 - reg_l2_loss: 0.0630 - loss: 1.1462 - learning_rate: 0.0090 - gradient_norm: 2.0001 - val_det_loss: 0.9926 - val_cls_loss: 0.5647 - val_box_loss: 0.0086 - val_reg_l2_loss: 0.0631 - val_loss: 1.0557
Epoch 2/50
187/187 [==============================] - 26s 139ms/step - det_loss: 0.7851 - cls_loss: 0.4114 - box_loss: 0.0075 - reg_l2_loss: 0.0632 - loss: 0.8483 - learning_rate: 0.0100 - gradient_norm: 2.6940 - val_det_loss: 0.8342 - val_cls_loss: 0.4875 - val_box_loss: 0.0069 - val_reg_l2_loss: 0.0633 - val_loss: 0.8975
Epoch 3/50
187/187 [==============================] - 25s 132ms/step - det_loss: 0.7291 - cls_loss: 0.3893 - box_loss: 0.0068 - reg_l2_loss: 0.0634 - loss: 0.7925 - learning_rate: 0.0099 - gradient_norm: 2.5211 - val_det_loss: 0.7079 - val_cls_loss: 0.3998 - val_box_loss: 0.0062 - val_reg_l2_loss: 0.0634 - val_loss: 0.7714
Epoch 4/50
187/187 [================

INFO:absl:use max_reduce for pre-nms topk.


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(24800, 7)
0/24800
DONE (t=0.05s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.87s).
Accumulating evaluation results...
DONE (t=0.18s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.259
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.558
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.213
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.338
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.173
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.340
 Average Recall

In [6]:
model.evaluate(test_data)


INFO:absl:use max_reduce for pre-nms topk.
2022-09-28 04:07:58.490120: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-28 04:07:58.490156: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


3/4 [=====================>........] - ETA: 0s

INFO:absl:use max_reduce for pre-nms topk.
2022-09-28 04:08:04.279254: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.75GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-28 04:08:04.279288: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.75GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


4/4 [==============================] - 12s 2s/step

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(25000, 7)
0/25000
DONE (t=0.05s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.60s).
Accumulating evaluation results...
DONE (t=0.18s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.358
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.637
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.245
 Average Recall     (AR) @[ IoU=0.50:0.95 |

{'AP': 0.35790035,
 'AP50': 0.6368026,
 'AP75': 0.3518009,
 'APs': 0.00017806508,
 'APm': 0.123154305,
 'APl': 0.44201422,
 'ARmax1': 0.24485743,
 'ARmax10': 0.43656155,
 'ARmax100': 0.48675883,
 'ARs': 0.014356435,
 'ARm': 0.2494898,
 'ARl': 0.5846347,
 'AP_/Person': 0.21012847,
 'AP_/Horse': 0.5056722}

In [8]:
model.export(export_dir='.')

INFO:tensorflow:Assets written to: /tmp/tmpkj4j57h5/assets


INFO:tensorflow:Assets written to: /tmp/tmpkj4j57h5/assets
2022-09-28 04:11:09.754018: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-09-28 04:11:15.436031: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-09-28 04:11:15.436054: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.


Estimated count of arithmetic ops: 1.752 G  ops, equivalently 0.876 G  MACs


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 0


Estimated count of arithmetic ops: 1.752 G  ops, equivalently 0.876 G  MACs
INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpn4pcgxp4/labelmap.txt.


INFO:tensorflow:Saving labels in /tmp/tmpn4pcgxp4/labelmap.txt.


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


In [9]:
model.evaluate_tflite('model.tflite', test_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


250/250 [==============================] - 28s 112ms/step

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(6250, 7)
0/6250
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.53s).
Accumulating evaluation results...
DONE (t=0.08s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.345
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.617
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.332
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.425
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.246
 Average Recall     (AR) @[ IoU=0.50:0

{'AP': 0.3451733,
 'AP50': 0.6173619,
 'AP75': 0.33190426,
 'APs': 0.00017721478,
 'APm': 0.120477505,
 'APl': 0.42545494,
 'ARmax1': 0.24624386,
 'ARmax10': 0.4082102,
 'ARmax100': 0.42124623,
 'ARs': 0.0024752475,
 'ARm': 0.1806187,
 'ARl': 0.5175608,
 'AP_/Person': 0.20550084,
 'AP_/Horse': 0.48484576}